In [1]:
import json
import torch

def get_ec_level(ec, level):
    return '.'.join(ec.split('.')[:level])

def ec_level(ec):
    return len(ec.split('.'))

with open('../data/ec/swissprot_ec_list_above_10.json') as f:
    ec_list = json.load(f)
print(len(ec_list), ec_list[:5])
all_level_ec_list = []
level_1_ec_list = list(set([get_ec_level(ec, 1) for ec in ec_list]))
level_2_ec_list = list(set([get_ec_level(ec, 2) for ec in ec_list]))
level_3_ec_list = list(set([get_ec_level(ec, 3) for ec in ec_list]))
level_4_ec_list = list(set([get_ec_level(ec, 4) for ec in ec_list]))
print(len(level_1_ec_list), len(level_2_ec_list), len(level_3_ec_list), len(level_4_ec_list))
with open('../data/ec/swissprot_ec_list_above_10_level_1.json', 'w') as f:
    json.dump(level_1_ec_list, f)
with open('../data/ec/swissprot_ec_list_above_10_level_2.json', 'w') as f:
    json.dump(level_2_ec_list, f)
with open('../data/ec/swissprot_ec_list_above_10_level_3.json', 'w') as f:
    json.dump(level_3_ec_list, f)
with open('../data/ec/swissprot_ec_list_above_10_level_4.json', 'w') as f:
    json.dump(level_4_ec_list, f)
all_level_ec_list = level_1_ec_list + level_2_ec_list + level_3_ec_list + level_4_ec_list
print(len(all_level_ec_list), all_level_ec_list[:10])
# Child Matrix: CM_ij = 1 if the jth EC number is a subclass of the ith EC number
CM = torch.zeros(len(all_level_ec_list), len(all_level_ec_list))
for i, ec in enumerate(all_level_ec_list):
    for j, ec2 in enumerate(all_level_ec_list):
        if get_ec_level(ec2, ec_level(ec)) == ec:
            CM[i, j] = 1
print(CM)
print(CM.sum())
print(CM.sum(0))
torch.save(CM, '../data/ec/CM_EC_all_level_label_above_10.pt')
with open('../data/ec/all_level_ec_list_above_10.json', 'w') as f:
    json.dump(all_level_ec_list, f)

1920 ['1.1.1.1', '1.1.1.10', '1.1.1.100', '1.1.1.102', '1.1.1.103']
7 67 211 1920
2205 ['6', '1', '5', '7', '2', '3', '4', '1.5', '1.14', '4.1']
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
tensor(8454.)
tensor([1., 1., 1.,  ..., 4., 4., 4.])


In [2]:
test = CM[:, -1]
test, test.sum()
one_indices = []
for i in range(len(test)):
    if test[i] == 1:
        one_indices.append(i)
print(one_indices)
print([all_level_ec_list[i] for i in one_indices])

[0, 17, 121, 2204]
['3', '3.6', '3.6.1', '3.6.1.75']


In [4]:
CM.sum(0).max()

tensor(4.)